<a href="https://colab.research.google.com/github/mltngpot/Describer/blob/main/Project3_HTMLParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import requests
from bs4 import BeautifulSoup

url = "https://www.purdue.edu/newsroom/purduetoday/2023/Q4/employees-can-now-connect-with-their-purdue-retirement-program-on-fidelity-netbenefits/"
#url = "https://www.purdue.edu/research/features/stories/new-ranking-demonstrates-purdue-prominence-in-ai-research-supercomputing/"
#url = "https://www.purdue.edu/newsroom/purduetoday/2023/Q4/stay-safe-and-injury-free-this-winter-top-tips-for-preventing-falls-and-walking-safely/"

response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the title using its ID
    title = soup.find('h1', id='main-heading')
    article_title = title.text.strip() if title else "Title not found"

    # Find the specific div with class "post-content__body"
    content_div = soup.find('div', class_='post-content__body')
    if content_div:
      # Extract text content from paragraphs and list items within this div
      paragraphs = [p.text.strip() for p in content_div.find_all(['p', 'li'])]

      # Extract image URLs within this div
      images = [img['src'] for img in content_div.find_all('img') if 'src' in img.attrs]

      # Combine all text
      article_text = "\n".join(paragraphs)
    else:
      print("The specified div with class 'post-content__body' was not found.")


    # Find the specific div with class "post-content__thumbnail" for images
    thumbnail_div = soup.find('div', class_='post-content__thumbnail')
    first_jpg_url = None
    if thumbnail_div:
        # Extract the first image tag
        img = thumbnail_div.find('img', class_='post-content__thumbnail--image')
        if img and 'srcset' in img.attrs:
            # Parse the srcset and find the first .jpg
            srcset = img['srcset']
            images = [img.strip().split(" ")[0] for img in srcset.split(",")]
            for image in images:
                if image.endswith('.jpg'):
                    first_jpg_url = image
                    break
        elif img and 'src' in img.attrs and img['src'].endswith('.jpg'):
            # Fallback to the src attribute if srcset is not present
            first_jpg_url = img['src']
    else:
        print("The specified div with class 'post-content__thumbnail' was not found.")

    # Prepare the output content
    output_content = f"Title:\n{article_title}\n\nArticle Text:\n{article_text}\n\nFirst JPG Image:\n{first_jpg_url}\n"

    # Print the results
    print(output_content)

    # Write the results to a text file
    with open("output.txt", "w", encoding="utf-8") as file:
        file.write(output_content)

    # Console outputs
    #print("Title:")
    #print(article_title)
    #print("Article Text:")
    #print(article_text)
    #print("\nThumbnail:")
    #print(first_jpg_url)

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")



Title:
Employees can now connect with their Purdue Retirement Program on Fidelity NetBenefits

Article Text:
Beginning Jan. 1, the web addresses NetBenefits.com/Purdue and NetBenefits.com/PG will bring employees directly to the Fidelity NetBenefits login screen, allowing for a more direct access experience to easily:
Access individual account information and personalized planning resources.
Learn and act on your financial wellness goals.
Get timely updates and alerts.
Schedule a one-on-one meeting with a Fidelity retirement planner.
Additional Fidelity account resources:
Download the NetBenefits mobile app to access retirement account information on the go.
Watch this short video to learn how to get started on NetBenefits.
Schedule online or call 800-642-7131 to make an in-person or virtual one-on-one appointment with a Fidelity retirement planner.

First JPG Image:
https://www.purdue.edu/newsroom/wp-content/uploads/2024/05/Winter-Purdue-2024_GMB5372.jpg

